In [1]:
import sys
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "autoencoders"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 텐서 플로의 정보 출력 억제하기
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # GPU 장치 지정

# tf.debugging.set_log_device_placement(True)   # 이거 쓰지 마셈 ㅈㄴ 출력 더러움

if not tf.config.list_physical_devices('GPU'):
    print("감지된 GPU가 없습니다. GPU가 없으면 LSTM과 CNN이 매우 느릴 수 있습니다.")

# 오토인코더와 GAN

##### 오토인코더 

- 단순히 입력을 출력으로 복사하는 방법을 배움

- 네트워크에 제약을 가해, 이 작업을 어렵게 만듦 (잠재 표현의 크기를 제한하거나, 입력에 잡음을 추가하고 원본 입력을 복원하도록 네트워크를 훈련)

- 오토인코더가 바로 복사하지 못하고, 데이터를 효율적으로 표현하는 방법을 배움

- 즉, 코딩은 일정 제약 조건하에, 항등 함수를 학습하려는 오토인코더의 노력으로 생긴 부산물

 

##### GAN

- 생성자와 판별자로 구성

- 생성자는 훈련 데이터와 비슷하게 보이는 데이터를 생성

- 판별자는 가짜 데이터와 진짜 데이터를 구별함

- 신경망이 훈련하는 동안 생성자와 판별자가 서로 경쟁함 -> 적대적 훈련

### 효율적인 데이터 표현하기

보통은 긴 시퀀스를 기억하기 어렵기 때문에 패턴을 찾는 것이 유용.

기억, 지각, 패턴 매칭 사이의 관계에 관한 연구 -> 

숙련된 체스 플레이어가 체스판을 5초만 보고도 전체 말의 위치를 외울 수 있다.

어떻게? 무작위로 놓였을 있을 때가 아니라 현실적인 위치에 있을 때.

체스 전문가라고 기억력이 뛰어난 것이 아님.

=> 즉 게임에 대한 경험 덕에 체스 패턴을 쉽게 보는 것.(패턴을 찾으면 정보를 효울적으로 저장가능)

In [3]:
import matplotlib.image as mpimg

filename = "auto_1.png"
images_path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
show_img = mpimg.imread(os.path.join(images_path, filename))
plt.figure(figsize=(20,10))
plt.axis("off")
plt.imshow(show_img)

FileNotFoundError: [Errno 2] No such file or directory: '.\\images\\autoencoders\\auto_1.png'

오토인코더는 입력을 받아 효율적인 내부 표현으로 바꾸고 입력과 가장 가까운 어떤 것을 출력함

오토인코더는 항상 두 부분으로 구성

- 입력을 내부 표현으로 바꾸는 인코더 (인지 네트워크)

- 내부 표현을 출력으로 바꾸는 디코더 (생성 네트워크)

In [ ]:
filename = "auto_2.png"
images_path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
show_img = mpimg.imread(os.path.join(images_path, filename))
plt.figure(figsize=(20,10))
plt.axis("off")
plt.imshow(show_img)

출력층의 뉴런 수가 입력 개수와 동일하다는 것을 제외하면, 오토인코더는 다층 퍼셉트론과 구조가 동일

- 뉴런 두 개로 구성된 하나의 은닉층 (인코더)

- 뉴런 세 개로 구성된 출력층 (디코더)

- 오토인코더가 입력을 재구성하기 때문에 출력을 재구성 이라고 부르고,

- 비용 함수는 재구성이 입력과 다를 때 모델에 벌점을 부과하는 재구성 손실을 포함함

내부의 표현이 입력 데이터보다 저차원(3차원 -> 2차원)이기 때문에, 

이런 오토인코더를 과소완전(undercomplete) 이라고 함

- 과소완전 오토인코더는 입력을 코딩으로 간단히 복사할 수 없으며, 
  
  입력과 똑같은 것을 출력하기 위한 방법을 찾아야함

-> 이를 통해 입력 데이터에서 가장 중요한 특성을 학습하도록 만듦

- 적어도 입력 데이터에 대해서는 복원(재구성)을 잘한다는 특징이 있음

### 과소완전 선형 오토인코더로 PCA 수행하기

오토인코더가 선형 활성화 함수만 사용하고 비용 함수가 MSE라면,

이는 결국 PCA(8장 참조)를 수행하는 것을 볼 수 있음.

28x28 흑백 이미지를 그리기 위한 유틸리티 함수:

In [ ]:
np.random.seed(4)

def generate_3d_data(m, w1=0.1, w2=0.3, noise=0.1):
    angles = np.random.rand(m) * 3 * np.pi / 2 - 0.5
    data = np.empty((m, 3))
    data[:, 0] = np.cos(angles) + np.sin(angles)/2 + noise * np.random.randn(m) / 2
    data[:, 1] = np.sin(angles) * 0.7 + noise * np.random.randn(m) / 2
    data[:, 2] = data[:, 0] * w1 + data[:, 1] * w2 + noise * np.random.randn(m)
    return data

X_train = generate_3d_data(60)
X_train = X_train - X_train.mean(axis=0, keepdims=0)

3D 데이터셋에 PCA를 적용해 2D에 투영하는 간단한 선형 오토인코더를 만듦.

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

encoder = keras.models.Sequential([keras.layers.Dense(2, input_shape=[3])])
decoder = keras.models.Sequential([keras.layers.Dense(3, input_shape=[2])])
autoencoder = keras.models.Sequential([encoder, decoder])

autoencoder.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1.5))

* 오토인코더를 인코더와 디코더 2개 컴포넌트로 구성. 둘 다 일반적인 Sequential 모델.

  오토인코더는 인코더 다음에 디코더가 뒤따르는 Sequential 모델.

* 오토인코더의 출력 개수 = 입력 개수와 동일 (즉, 3개).

* 단순한 PCA를 수행하기 위해선 활성화 함수를 사용하지 않으며(즉, 모든 뉴런이 선형). 비용함수는 MSE

3D 데이터셋에 훈련 후 모델을 사용해 동일한 데이터셋을 인코딩 (= 2D로 투영) :

In [ ]:
history = autoencoder.fit(X_train, X_train, epochs=20)

In [ ]:
codings = encoder.predict(X_train)

In [ ]:
fig = plt.figure(figsize=(4,3))
plt.plot(codings[:,0], codings[:, 1], "b.")
plt.xlabel("$z_1$", fontsize=18)
plt.ylabel("$z_2$", fontsize=18, rotation=0)
plt.grid(True)
save_fig("linear_autoencoder_pca_plot")
plt.show()

In [ ]:
filename = "auto_3.png"
images_path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
show_img = mpimg.imread(os.path.join(images_path, filename))
plt.figure(figsize=(20,10))
plt.axis("off")
plt.imshow(show_img)

오토인코더는 PCA처럼 데이터에 있는 분산이 가능한 많이 보존되도록 데이터를 투영할 최상의 2D 평면을 찾음

### 적층 오토인코더

은닉층을 여러 개 가지는 오토인코더

- 층을 더 추가하여 더 복잡한 코딩을 학습할 수 있음

- 오토인코더가 너무 강력해지지 않도록 주의
  - 인코더가 너무 강력해서 각각의 입력 데이터를 임의의 한 숫자로 매핑하도록 학습했다고 가정
  - 훈련 데이터를 완벽히 재구성하겠지만, 유용한 데이터 표현을 학습하지 못할 것

In [ ]:
filename = "auto_4.png"
images_path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
show_img = mpimg.imread(os.path.join(images_path, filename))
plt.figure(figsize=(20,10))
plt.axis("off")
plt.imshow(show_img)

- 적층 오토인코더는 전형적으로 가운데 은닉층(코딩 층)을 기준으로 대칭

- 위 예시는 입력 784개, 뉴런 300개로 된 은닉층, 뉴런 150개로 된 가운데 은닉층, 뉴런 300개로 된 은닉층, 뉴런 784개로 된 출력층

MNIST 데이터셋을 사용합니다:

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full.astype(np.float32) / 255
X_test = X_test.astype(np.float32) / 255
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

#### 케라스를 사용해 적층 오토인코더 구현하기

3개의 은닉층과 1개의 출력층(즉, 두 개를 적층)을 가진 적층 오토인코더를 만들어 보겠습니다.

In [ ]:
def rounded_accuracy(y_true, y_pred):
    return keras.metrics.binary_accuracy(tf.round(y_true), tf.round(y_pred))

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

stacked_encoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(30, activation="selu"),
])
stacked_decoder = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[30]),
    keras.layers.Dense(28 * 28, activation="sigmoid"),
    keras.layers.Reshape([28, 28])
])
stacked_ae = keras.models.Sequential([stacked_encoder, stacked_decoder])
stacked_ae.compile(loss="binary_crossentropy",
                   optimizer=keras.optimizers.SGD(learning_rate=1.5), metrics=[rounded_accuracy])
history = stacked_ae.fit(X_train, X_train, epochs=20,
                         validation_data=(X_valid, X_valid))

- 오토인코더 모델을 인코더와 디코더 두 모델로 나눔

- 인코더는 28x28 픽셀의 흑백 이미지를 받고, 이미지를 784 크기의 벡터로 표현하기 위해 펼침
  
  SELU 활성화 함수를 사용하는 Dense 층 두 개에 통과시킴 
  
  (르쿤 정규분포 초기화를 추가할 수 있으나 차이가 크지 않음 네트워크가 깊지 않아서)

  각 입력 이미지에 대해 인코더는 크기가 30인 벡터를 출력함

- 디코더는 크기가 30인 코딩을 받아, 크기가 커지는 Dense 층 두 개에 통과시킴

  최종 벡터를 28x28 배열로 변경하여 디코더의 출력이 인코더의 입력과 동일한 크기가 되도록 함

- 적층 인코더를 컴파일할 때 평균 제곱 오차 대신 이진 크로스 엔트로피 손실을 사용

  - 재구성 작업을 다중 레이블 이진 분류 문제로 다루는 것
  
  - 각 필셀의 강도는 픽셀이 검정일 확률을 나타냄

- X_train을 입력과 타깃으로 사용해 모델을 훈련함 (X_valid를 검증 입력과 검증 타킷으로 사용)

#### 재구성 시각화

오토인코더가 잘 훈련되었는지 확인하는 방법은 입력과 출력을 비교하는 것

- 입력과 출력의 차이가 크지 않아야 함

In [ ]:
def plot_image(image):
    plt.imshow(image, cmap="binary")
    plt.axis("off")

def show_reconstructions(model, images=X_valid, n_images=5):
    reconstructions = model.predict(images[:n_images])
    fig = plt.figure(figsize=(n_images * 1.5, 3))
    for image_index in range(n_images):
        plt.subplot(2, n_images, 1 + image_index)
        plot_image(images[image_index])
        plt.subplot(2, n_images, 1 + n_images + image_index)
        plot_image(reconstructions[image_index])

In [ ]:
show_reconstructions(stacked_ae)
save_fig("reconstruction_plot")

- 재구성된 이미지를 식별할 수는 있지만, 정보를 많이 읽은 모습

- 모델을 더 훈련하고 인코더/디코더 층을 늘리거나 코딩을 늘리면 나아질 수 있으나, 네트워크가 너무 강력하면 데이터에서 유익한 패턴을 학습하지 못하고 완벽한 재구성 이미지를 만드려고할 것

#### 패션 MNIST 데이터셋 시각화

훈련한 적층 오토인코더 모델을 사용해, 데이터 셋의 차원을 축소할 수 있음

- 시각화 입장에서 보면, 다른 차원 축소 알고리즘만큼의 성능은 아니지만

- 샘플과 특성이 많은 대용량 데이터셋을 다룰 수 있음

- 오토인코더를 사용해 적절히 차원을 축소하고, 다른 차원 축소 알고리즘을 사용해 시각화하는 것도 방법

패션 MNIST 데이터셋 시각화

- 적층 오토인코더의 인코더 모델을 사용해 차원을 30으로 줄이고,

- t-SNE 알고리즘을 구현한 사이킷런 클래스를 통해 차원을 2까지 줄임

In [ ]:
np.random.seed(42)

from sklearn.manifold import TSNE

X_valid_compressed = stacked_encoder.predict(X_valid)
tsne = TSNE()
X_valid_2D = tsne.fit_transform(X_valid_compressed)
X_valid_2D = (X_valid_2D - X_valid_2D.min()) / (X_valid_2D.max() - X_valid_2D.min())

##### **왜 대용량 고차원 데이터셋을 차원 축소하는 데 적합한가?**

8장 차원축소 장에서 잠깐 다루었던 LLE, Isomap 등은 각 샘플 별로 근방에 있는 데이터를 사용하는 neighborhood 기반 방법임

-> 즉, 가까운 애들이 manifold 상에서도 가까울 것이라고 가정하는 것

-> 하지만, 의미적으로 가까울 것이라고 생각하는 샘플들이 가깝지 않을 수 있음

  (고차원 데이터 간의 유클리디안 거리 != 유의미한 거리)

-> 데이터 공간이 고차원일 수록, 매니폴드를 찾기 어려울 수록

-> neighborhood방식이 아닌 DNN 기반의 오토인코더가 유용할 수 있음

In [ ]:
plt.scatter(X_valid_2D[:, 0], X_valid_2D[:, 1], c=y_valid, s=10, cmap="tab10")
plt.axis("off")
plt.show()

In [ ]:
# https://scikit-learn.org/stable/auto_examples/manifold/plot_lle_digits.html 참고
plt.figure(figsize=(10, 8))
cmap = plt.cm.tab10
plt.scatter(X_valid_2D[:, 0], X_valid_2D[:, 1], c=y_valid, s=10, cmap=cmap)
image_positions = np.array([[1., 1.]])
for index, position in enumerate(X_valid_2D):
    dist = np.sum((position - image_positions) ** 2, axis=1)
    if np.min(dist) > 0.02: # if far enough from other images
        image_positions = np.r_[image_positions, [position]]
        imagebox = mpl.offsetbox.AnnotationBbox(
            mpl.offsetbox.OffsetImage(X_valid[index], cmap="binary"),
            position, bboxprops={"edgecolor": cmap(y_valid[index]), "lw": 2})
        plt.gca().add_artist(imagebox)
plt.axis("off")
save_fig("fashion_mnist_visualization_plot")
plt.show()

#### 적층 오토인코더를 사용한 비지도 사전훈련

레이블된 훈련 데이터가 많지 않을때 적층 오토인코더를 통해 훈련하기 

- 대부분 레이블되지 않은 대량의 데이터셋을, 먼저 전체 데이터셋을 사용해 적층 오토인코더를 훈련

- 그 다음, 오토인코더의 하위층을 재사용해 실제 문제를 해결하기 위한 신경망을 만들고, 레이블된 데이터를 사용해 훈련

구현도 아래와 같이 가능

1. 레이블된 것, 레이블되지 않은 모든 훈련 데이터를 사용해 오토인코더를 훈련

2. 그 후 인코더 층을 재사용하여 새로운 신경망을 만듦

3. 레이블된 데이터를 통해 훈련

#### 가중치 묶기

위 예제처럼, 오토인코더가 대칭일 땐, 디코더의 가중치와 인코더의 가중치를 묶는 것이 일반적

- 모델에 있는 가중치의 수를 절반으로 줄여 훈련 속도를 높이고, 과대적합의 위험을 줄임

 

어떤 오토인코더가 $N$ 개 층을 갖고, $W_L$ 이 $L$ 번째 층의 가중치를 나타낼 때

- (1은 첫 번째 은닉층, $N/2$ 은 코딩 층, $N$ 은 출력층)

- 디코더 층의 가중치는 $W_{N-L}+1 = W_L^T (L = 1,2, ..., N/2)$

##### **케라스의 사용자 정의 층을 만들어 층 간 가중치 묶기 :**

In [ ]:
class DenseTranspose(keras.layers.Layer):
    def __init__(self, dense, activation=None, **kwargs):
        self.dense = dense
        self.activation = keras.activations.get(activation)
        super().__init__(**kwargs)
    def build(self, batch_input_shape):
        self.biases = self.add_weight(name="bias",
                                      shape=[self.dense.input_shape[-1]],
                                      initializer="zeros")
        super().build(batch_input_shape)
    def call(self, inputs):
        z = tf.matmul(inputs, self.dense.weights[0], transpose_b=True)  
        # transpose_b=True로 지정하는 것과 동일하나 matmul()이 더 효율적
        
        return self.activation(z + self.biases)

- 일반적인 Dense 층과 비슷하지만, 다른 Dense 층의 전치된 가중치를 사용함

- 편향 벡터는 독자적으로 사용

##### **위 사용자 정의층을 사용하여 적층 오토인코더 구성 :**

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

dense_1 = keras.layers.Dense(100, activation="selu")
dense_2 = keras.layers.Dense(30, activation="selu")

tied_encoder = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    dense_1,
    dense_2
])

tied_decoder = keras.models.Sequential([
    DenseTranspose(dense_2, activation="selu"),
    DenseTranspose(dense_1, activation="sigmoid"),
    keras.layers.Reshape([28, 28])
])

tied_ae = keras.models.Sequential([tied_encoder, tied_decoder])

tied_ae.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1.5), metrics=[rounded_accuracy])
history = tied_ae.fit(X_train, X_train, epochs=10,
                      validation_data=(X_valid, X_valid))

- 이 모델은 절반의 파라미터로 이전 모델보다 약간 낮은 내구성 오차를 달성함

In [ ]:
show_reconstructions(tied_ae)
plt.show()

#### 한 번에 오토인코더 한 개씩 훈련하기

한 번에 전체 오토인코더를 훈련하지 않고, 오토인코더 하나를 훈련하고 이를 쌓아올려서 한 개의 적층 오토인코더를 만들 수 있음

- 현재는 많이 사용 x

- '탐욕적 방식의 층별 훈련' 에 대한 논문에 등장

In [ ]:
filename = "auto_5.png"
images_path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
show_img = mpimg.imread(os.path.join(images_path, filename))
plt.figure(figsize=(20,10))
plt.axis("off")
plt.imshow(show_img)

- 단계 1 에서 첫 번째 오토인코더는 입력을 재구성하도록 학습

- 이 오토인코더를 사용해 전체 훈련 세트는 인코딩하여 (압축된) 새 훈련 세트를 만듦

- 새로운 훈련 세트에서 두 번째 오토인코더를 훈련 (단계 2)

- 마지막으로 모든 오토인코더를 사용해 전체 네트워크를 만듦 (단계 3, 각 오토인코더의 은닉층을 먼저 쌓고, 그 다음 출력층을 (반대로) 쌓음) -> 최종 적층 오토인코더

In [ ]:
def train_autoencoder(n_neurons, X_train, X_valid, loss, optimizer,
                      n_epochs=10, output_activation=None, metrics=None):
    n_inputs = X_train.shape[-1]
    encoder = keras.models.Sequential([
        keras.layers.Dense(n_neurons, activation="selu", input_shape=[n_inputs])
    ])
    decoder = keras.models.Sequential([
        keras.layers.Dense(n_inputs, activation=output_activation),
    ])
    autoencoder = keras.models.Sequential([encoder, decoder])
    autoencoder.compile(optimizer, loss, metrics=metrics)
    autoencoder.fit(X_train, X_train, epochs=n_epochs,
                    validation_data=(X_valid, X_valid))
    return encoder, decoder, encoder(X_train), encoder(X_valid)

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

K = keras.backend
X_train_flat = K.batch_flatten(X_train) # equivalent to .reshape(-1, 28 * 28)
X_valid_flat = K.batch_flatten(X_valid)
enc1, dec1, X_train_enc1, X_valid_enc1 = train_autoencoder(
    100, X_train_flat, X_valid_flat, "binary_crossentropy",
    keras.optimizers.SGD(learning_rate=1.5), output_activation="sigmoid",
    metrics=[rounded_accuracy])
enc2, dec2, _, _ = train_autoencoder(
    30, X_train_enc1, X_valid_enc1, "mse", keras.optimizers.SGD(learning_rate=0.05),
    output_activation="selu")

In [ ]:
stacked_ae_1_by_1 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    enc1, enc2, dec2, dec1,
    keras.layers.Reshape([28, 28])
])

In [ ]:
show_reconstructions(stacked_ae_1_by_1)
plt.show()

In [ ]:
stacked_ae_1_by_1.compile(loss="binary_crossentropy",
                          optimizer=keras.optimizers.SGD(learning_rate=0.1), metrics=[rounded_accuracy])
history = stacked_ae_1_by_1.fit(X_train, X_train, epochs=10,
                                validation_data=(X_valid, X_valid))

In [ ]:
show_reconstructions(stacked_ae_1_by_1)
plt.show()

### 합성곱 오토인코더

### 순환 오토인코더

### 잡음 제거 오토인코더

### 희소 오토인코더

### 변이형 오토인코더

#### 패션 MNIST 이미지 생성하기

### 생성적 적대 신경망

##### **생성자**

##### **판별자**

#### GAN 훈련의 어려움

#### 심층 합성곱 GAN

#### ProGAN

##### **미니배치 표준편차 층**

##### **동일한 학습 속도**

##### **픽셀별 정규화 층**

#### StyleGAN

##### **합성 네트워크**